In [1]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os
import skimage

from PIL import Image
from tqdm import tqdm

In [4]:
folder = '../../allwavs/allvowl/corrected/extracted/trimmed'
files = os.listdir(folder)

In [5]:
n_fft = 512
hop_length = 512
n_mels = 128

# https://stackoverflow.com/questions/57204349#57204349
def scale_minmax(X, min=0.0, max=1.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled

def spectrogram_image(y, sr, out, hop_length, n_mels):
    # use log-melspectrogram
    mels = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length)
    mels = np.log(mels + 1e-9) # add small number to avoid log(0)

    img = scale_minmax(mels, 0, 255).astype(np.uint8)
    img = np.flip(img, axis=0) # put low frequencies at the bottom in image
    img = 255-img
    skimage.io.imsave(out, img)

for file in tqdm(files):
    if file == 'img' or file == 'trimmed':
        continue
    y, sr = librosa.load(folder + '/' + file)
    spectrogram_image(y=y, sr=sr, out=folder + '/img/' + file.replace('.wav', '__mel.png'), hop_length=hop_length, n_mels=n_mels)
    
    plt.close('all')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 5575/5575 [04:28<00:00, 20.73it/s]
